TP

### 1. Génération des données du problème. <br>
#### a) Générez les données du problème

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cvx
import time
n = 200 # number of examples (you can try with n = 1000 and n = 5000)
p = 2*n # dimensionality of the problem
k = 5 # number of active variables
np.random.seed(0)
X = np.random.randn(n,p) # creating features and normalizing them
X = (X - np.mean(X,axis = 0))/np.std(X,axis = 0)
t = np.arange(0,p)/(p-1); # bulding the variance matrix !
S = np.zeros((p,p))
nn = 0.00001
for i in range(p):
    S[i,:] = np.exp(-(t-t[i])**2/nn);
X = X@(S**.5)
X = X/np.linalg.norm(X,axis=0)
ind = np.random.choice(p, k, replace=False) # generating optimal weights
print(f"index: {ind}")
weights = np.random.randn(k)
weights += 0.1+np.sign(weights) # to get large enough weight
wopt = np.zeros(p)
wopt[ind] = weights
rsnr = 2 # generating output by X@w + noise
z = X[:,ind]@weights
stdnoise = np.std(z)/rsnr
y = z + stdnoise*np.random.randn(n)

index: [309 390  55  82 329]


#### b) Vérifiez que les données ont bien les propriétés attendues

In [2]:
print(f"Taille de X: {X.shape}")
print(f"Taille de Y: {y.shape}")
print(f"Taille de S: {S.shape}")
print(f"weights: {weights}")

Taille de X: (200, 400)
Taille de Y: (200,)
Taille de S: (400, 400)
weights: [-1.12180992  1.31787313 -0.92626818  2.65803182 -1.51223091]


In [3]:
print(f"X:\n {X[:5,:]}")

X:
 [[ 0.14467085  0.16433926  0.19901953 ...  0.14290542  0.1631462
   0.13879548]
 [-0.05527668 -0.03926178 -0.01261321 ... -0.08845567 -0.09053221
  -0.08149685]
 [ 0.1156909   0.09286922  0.05119743 ... -0.06945269 -0.09496954
  -0.05753581]
 [-0.091624   -0.05659364  0.00736161 ... -0.00039534 -0.04531035
  -0.06724717]
 [ 0.00719499 -0.03644546 -0.09217101 ...  0.00423391 -0.07740297
  -0.11560728]]


#### c) Calculez l’erreur de généralisation "in sample" de la méthode des moindres carrés

In [4]:
b_ls = np.linalg.solve(X.T@X,X.T@y)
e_ls = np.sum((X@b_ls-z)**2)
print("Test error for the LS regression: {:0.4f}".format(e_ls))

Test error for the LS regression: 2.7231


#### d) Écrire une fonction Eval_coef(X,z,coeff), qui calcule l’erreur de généralisation "insample"

In [5]:
def Eval_coef(X,z,coeff):
    e = np.sum((X@coeff - z)**2)
    return e
print(f"Test error for the LS regression (utilisant mon Eval_coef function): {Eval_coef(X,z,b_ls):0.4f}")

Test error for the LS regression (utilisant mon Eval_coef function): 2.7231


### 2. Différentes manières de résoudre le problème du Lasso <br>
#### a) Écrire un programme CVX résolvant, pour λ = 10−3n.

In [24]:
λ = 1e-3*n # le paramètre de régularisation
b = cvx.Variable(p) # le vecteur des coefficients, il sert de variable d'optimisation
obj = cvx.Minimize(0.5*cvx.sum_squares(X@b - y) + λ*cvx.norm(b, 1)) # la fonction objectif à minimiser
prob = cvx.Problem(obj) # le problème d'optimisation
start_time = time.time()
prob.solve(solver=cvx.SCS, eps=1e-5) # résolution du problème avec le solveur SCS
end_time = time.time()

# b) Vérifiez que la solution obtenue est meilleure que celle des moindres carrés
b_ls = np.linalg.solve(X.T@X,X.T@y) # recalcul des coefficients de la régression des moindres carrés
e_ls = np.sum((X@b_ls-z)**2) 
np.set_printoptions(formatter={'float': '{: 0.3f}'.format}) # pour afficher les nombres avec 3 décimales
print("In sample error for the LS regression: {:0.4f}".format(e_ls))

e_la = np.sum((X@b.value-z)**2)
print("In sample error for the Lasso regression: {:0.4f}".format(e_la))

In sample error for the LS regression: 2.7231
In sample error for the Lasso regression: 0.3323


In [7]:
print(f"Alors la methode lasso est bien meilleure que les moindres carrés. avec une erreur de {e_ls-e_la:0.4f} de moins")
print(f"Le temps d'execution est de {end_time - start_time:0.4f} secondes")
print(f"Le nombre de coefficients non nuls est de {np.sum(np.abs(b.value)>1e-4)}")
print(f"Les coefficients non nuls sont aux indices: {np.where(np.abs(b.value)>1e-4)[0]}")
print(f"Les coefficients estimés sont: {b.value[np.where(np.abs(b.value)>1e-4)]}")

Alors la methode lasso est bien meilleure que les moindres carrés. avec une erreur de 2.3908 de moins
Le temps d'execution est de 0.1795 secondes
Le nombre de coefficients non nuls est de 24
Les coefficients non nuls sont aux indices: [ 18  22  42  43  47  55  70  82  93 118 178 190 191 261 275 291 302 309
 310 329 389 390 393 397]
Les coefficients estimés sont: [-0.042  0.081 -0.003 -0.026 -0.018 -0.697 -0.002  2.623 -0.063  0.087
  0.030 -0.192 -0.002 -0.023  0.045 -0.062  0.005 -0.683 -0.018 -1.442
  0.008  1.074  0.036 -0.019]


#### b) Écrire un programme CVX résolvant la formulation suivant de Lasso, avec une valeur <br> de t permettant d’obtenir les mêmes résultats que le problème précédent.

In [8]:
t = np.sum(np.abs(b.value))
b1 = cvx.Variable(p)
obj1 = cvx.Minimize(cvx.sum_squares(X@b1 - y))
c1 = [cvx.norm(b1, 1) <= t]
prob1 = cvx.Problem(obj1, c1)
start_time = time.time()
prob1.solve()
end_time = time.time()
e_la1 = np.sum((X@b1.value - z)**2)
print("In sample error for the Lasso regression (formulation contrainte): {:0.4f}".format(e_la1))

In sample error for the Lasso regression (formulation contrainte): 0.3322


In [9]:
print(f"Avec t = {t:0.4f}, on retrouve bien la même erreur que la formulation precedente avec le lambda")
print("Temps prit pour résoudre le problème Lasso: {:0.4f} secondes".format(end_time - start_time))
print("Le nombre de coefficients non nuls est de {:d}".format(np.sum(np.abs(b1.value)>1e-4)))
print(f"Les coefficients non nuls sont aux indices: {np.where(np.abs(b1.value)>1e-4)[0]}")

Avec t = 7.2826, on retrouve bien la même erreur que la formulation precedente avec le lambda
Temps prit pour résoudre le problème Lasso: 0.5309 secondes
Le nombre de coefficients non nuls est de 24
Les coefficients non nuls sont aux indices: [ 18  22  42  43  47  55  70  82  93 118 178 190 191 261 275 291 302 309
 310 329 389 390 393 397]


#### c) Écrire un programme CVX résolvant la formulation suivant de Lasso, avec une valeur <br> de ε permettant d’obtenir les mêmes résultats que le problème précédent.

In [10]:
ε = np.sum((X@b1.value - y)**2)
b2 = cvx.Variable(p)
obj2 = cvx.Minimize(cvx.norm(b2, 1))
c2 = [cvx.sum_squares(X@b2 - y) <= ε]
prob2 = cvx.Problem(obj2, c2)
start_time = time.time()
prob2.solve()
end_time = time.time()
e_la2 = np.sum((X@b2.value - z)**2)
print("In sample error for the Lasso regression (formulation contrainte): {:0.4f}".format(e_la2))
print(f"Temps de calcul pour la formulation pénalisée: {end_time - start_time:0.4f} secondes")


In sample error for the Lasso regression (formulation contrainte): 0.3322
Temps de calcul pour la formulation pénalisée: 0.5832 secondes


In [11]:
### ANALYSE DES RÉSULTATS
# avec lambda = 1e-3*n, t, ε, on retrouve bien les mêmes erreurs pour les 3 formulations du Lasso.
# Le temps de calcul est plus élevé pour la formulation contrainte avec ε.
# Le Lasso est bien meilleur que les moindres carrés en terme d'erreur.


In [33]:
λ = 1 # le paramètre de régularisation
b_ridge = cvx.Variable(p) # le vecteur des coefficients, il sert de variable d'optimisation
obj = cvx.Minimize(0.5*cvx.sum_squares(X@b_ridge - y) + λ*cvx.sum_squares(b_ridge))
prob = cvx.Problem(obj) # le problème d'optimisation
start_time = time.time()
prob.solve(solver=cvx.SCS, eps=1e-5) # résolution du problème avec le solveur SCS
end_time = time.time()

# b) Vérifiez que la solution obtenue est meilleure que celle des moindres carrés
b_ls = np.linalg.solve(X.T@X,X.T@y) # recalcul des coefficients de la régression des moindres carrés
e_ls = np.sum((X@b_ls-z)**2) 
np.set_printoptions(formatter={'float': '{: 0.3f}'.format}) # pour afficher les nombres avec 3 décimales
print("In sample error for the LS regression: {:0.4f}".format(e_ls))

e_ridge = np.sum((X@b_ridge.value-z)**2)
print("In sample error for the ridge regression: {:0.4f}".format(e_ridge))

In sample error for the LS regression: 2.7231
In sample error for the ridge regression: 2.3094


In [34]:
grad = (X.T @ X) @ b.value - X.T @ y + 2 * λ * b.value
np.linalg.norm(grad)


np.float64(6.536919119380083)

In [47]:
def my_ridge(X, y, lam):
    n, p = X.shape
    A = X.T @ X + 2.0 * lam * np.eye(p) 
    rhs = X.T @ y                         
    b = np.linalg.solve(A, rhs)
    return b

b_ridge = my_ridge(X, y, λ)
e_ridge = np.sum((X @ b_ridge - z) ** 2)
print("ridge regression: {:0.4f}".format(e_ridge))
grad = (X.T @ X) @ b_ridge - X.T @ y + 2 * λ * b_ridge
np.linalg.norm(grad)

ridge regression: 2.3094


np.float64(3.665773238953586e-15)

In [45]:
# import numpy as np

# def check_KKT(X, y, beta, t, tol=1e-6):
#     # stationnarité : X^T(Xβ - y) + μβ = 0
#     residual = X @ beta - y
#     grad = X.T @ residual   # gradient OLS
#     norm_sq = 0.5 * np.linalg.norm(beta)**2

#     # Calcul du multiplicateur de Lagrange μ
#     if np.allclose(norm_sq, t, atol=tol):
#         # contrainte active → μ > 0
#         mu = - (grad @ beta) / (np.linalg.norm(beta)**2 + 1e-12)
#     else:
#         # contrainte inactive → μ = 0
#         mu = 0.0

#     # Vérification des KKT
#     stationarity = np.linalg.norm(grad + mu * beta)
#     complementarity = mu * (norm_sq - t)

#     return {
#         "norm_sq": norm_sq,
#         "mu": mu,
#         "stationarity": stationarity,
#         "complementarity": complementarity
#     }


# check = check_KKT(X, y, b_ridge, t=1e6)
# print(check)


In [63]:
t = 2.507

b_ridge = cvx.Variable(p)
constraint = [0.5 * cvx.sum_squares(b_ridge) <= t]
obj = cvx.Minimize(cvx.sum_squares(X @ b_ridge - y))
prob = cvx.Problem(obj, constraints=constraint)
prob.solve(solver=cvx.SCS, eps=1e-5)

beta_hat = b_ridge.value
print(f"norme de la solution: {np.linalg.norm(beta_hat)**2:0.4f} (doit être <= {2*t:0.4f})")
print(f"respecte la contrainte: {0.5 * np.linalg.norm(beta_hat)**2 <= t + 1e-4}")

## KKT
grad = X.T @ (X @ beta_hat - y)

# multiplicateur de Lagrange μ associé à la contrainte
lambda_hat = constraint[0].dual_value

# stationnarité
stationnarity = grad + 2 * lambda_hat * beta_hat
print(f"||stationnarité||: {np.linalg.norm(stationnarity):.4e}")
print(f"max violation stationnarité: {np.max(np.abs(stationnarity)):.4e}")

print(f"complementarité: {lambda_hat * (np.linalg.norm(beta_hat)**2 - t)}")
print("lambda_hat >= 0 ?", lambda_hat >= -1e-6)

norme de la solution: 5.0140 (doit être <= 5.0140)
respecte la contrainte: True
||stationnarité||: 1.7448e+00
max violation stationnarité: 7.6730e-01
complementarité: [ 1.302]
lambda_hat >= 0 ? [ True]
